In [ ]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port', '0'). \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [ ]:
spark

In [ ]:
orders_base = spark.sparkContext.textFile("/public/trendytech/orders/orders_1gb.csv")

In [ ]:
orders_mapped = orders_base.map(lambda x: (x.split(",")[2], x.split(",")[3]))

In [ ]:
orders_mapped.take(5)

In [ ]:
customers_base = spark.sparkContext.textFile("/public/trendytech/retail_db/customers")

In [ ]:
customers_mapped = customers_base.map(lambda x: (x.split(",")[0], x.split(",")[8]))

In [ ]:
customers_mapped.take(5)

In [ ]:
customers_broadcast = spark.sparkContext.broadcast(customers_mapped.collect())

In [ ]:
def getPinCode(customerID):
    try:
        for customer in customers_broadcast .value:
            if customer[0] == str(customerID):
                return (customer[0],customer[1])
        return -1    
    except:
        return -1

In [ ]:
joined_rdd = orders_mapped.map(lambda x : (getPinCode(int(x[0])), x[1]))

In [ ]:
joined_rdd.saveAsTextFile("data/broadcastresults")

In [ ]:
getPinCode(1)